In [2]:
import os
import pandas as pd
import glob
import yaml
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data_path = 'data'
performance_path = os.path.join(data_path, 'performance', 'final_classifier_grid')
config_path = os.path.join(data_path, 'config', 'grid', 'final_classifier')

In [4]:
figure_path = os.path.join(data_path, 'figures')
os.makedirs(figure_path, exist_ok = True)

In [5]:
def path_split(x):
    x = x.replace('/', ' ').split()
    return x

In [38]:
files_eval = glob.glob(os.path.join(performance_path, '**', '*_eval.csv'), recursive=True)
df_eval = pd.concat({(file, path_split(file)[-4], path_split(file)[-3], path_split(file)[-2]): 
                        pd.read_csv(file) for file in files_eval})
df_eval = df_eval.rename_axis(index = ['file_path', 'outcome', 'sensitive_variable', 'experiment_id', 'index'])
df_eval = df_eval.reset_index([0, 1, 2, 3])
df_eval['experiment_id'] = df_eval['experiment_id'].astype(int)

In [39]:
df_eval.head()

,file_path,outcome,sensitive_variable,experiment_id,metric,phase,epoch,performance
index,,,,,,,,
0,data/performance/final_classifier_grid/los/age...,los,age,0,auc,val,0,0.840372
1,data/performance/final_classifier_grid/los/age...,los,age,0,auprc,val,0,0.587191
2,data/performance/final_classifier_grid/los/age...,los,age,0,brier,val,0,0.122572
3,data/performance/final_classifier_grid/los/age...,los,age,0,classification,val,0,0.397314
4,data/performance/final_classifier_grid/los/age...,los,age,0,classification_cf,val,0,0.264394


In [22]:
files_config = glob.glob(os.path.join(config_path, '**', 'config.csv'), recursive=True)

In [31]:
df_config = pd.concat({(file, path_split(file)[-3], path_split(file)[-2]): 
                pd.read_csv(file, dtype = {'id' : int}) for file in files_config})

In [32]:
df_config = df_config.rename_axis(index = ['file_path', 'outcome', 'sensitive_variable', 'index'])
df_config = df_config.rename(columns = {'id' : 'experiment_id'})
df_config = df_config.reset_index([0, 1, 2])

In [33]:
df_config.head()

,file_path,outcome,sensitive_variable,experiment_id,cf_gradients,lambda_clp,lambda_clp_entropy,lambda_final_classifier_cf,lr_final_classifier,num_epochs,num_samples_eval,weighted
index,,,,,,,,,,,,
0,data/config/grid/final_classifier/los/age/conf...,los,age,0,False,0.0,0.0,10.0,0.0010,20,1,False
1,data/config/grid/final_classifier/los/age/conf...,los,age,1,True,0.1,0.0,0.1,0.0100,20,1,False
2,data/config/grid/final_classifier/los/age/conf...,los,age,2,True,0.1,0.0,1.0,0.0100,20,1,False
3,data/config/grid/final_classifier/los/age/conf...,los,age,3,True,0.0,0.0,1.0,0.0001,20,1,False
4,data/config/grid/final_classifier/los/age/conf...,los,age,4,False,0.1,0.0,1.0,0.0010,20,1,False


In [34]:
df_eval.head()

,file_path,outcome,sensitive_variable,experiment_id,metric,phase,epoch,performance
index,,,,,,,,
0,data/performance/final_classifier_grid/los/age...,los,age,0,auc,val,0,0.840372
1,data/performance/final_classifier_grid/los/age...,los,age,0,auprc,val,0,0.587191
2,data/performance/final_classifier_grid/los/age...,los,age,0,brier,val,0,0.122572
3,data/performance/final_classifier_grid/los/age...,los,age,0,classification,val,0,0.397314
4,data/performance/final_classifier_grid/los/age...,los,age,0,classification_cf,val,0,0.264394


In [40]:
print(df_eval.apply(lambda x: type(x.array[0])))
print(df_config_temp.apply(lambda x: type(x.array[0])))

file_path               <class 'str'>
outcome                 <class 'str'>
sensitive_variable      <class 'str'>
experiment_id           <class 'int'>
metric                  <class 'str'>
phase                   <class 'str'>
epoch                   <class 'int'>
performance           <class 'float'>
dtype: object
file_path                       <class 'str'>
outcome                         <class 'str'>
sensitive_variable              <class 'str'>
experiment_id                   <class 'int'>
cf_gradients                   <class 'bool'>
lambda_clp                    <class 'float'>
lambda_clp_entropy            <class 'float'>
lambda_final_classifier_cf    <class 'float'>
lr_final_classifier           <class 'float'>
num_epochs                      <class 'int'>
num_samples_eval                <class 'int'>
weighted                       <class 'bool'>
dtype: object


In [108]:
df_config_temp = df_config.loc[(df_config.cf_gradients == False) &
                               (df_config.lambda_final_classifier_cf == 1.0) &
                               (df_config.lr_final_classifier == 1e-3)
                              ]
df_summary = pd.merge(df_eval, df_config_temp, how = 'inner', on = ['experiment_id', 'outcome', 'sensitive_variable'])
df_summary = df_summary.loc[df_summary.phase == 'test']
temp = df_summary.groupby(['outcome', 'sensitive_variable', 'metric', 'lambda_clp'])[['performance']] \
            .agg(lambda x: x). \
            reset_index()
temp = temp.loc[(temp.outcome == 'los') & (temp.sensitive_variable == 'age')]
temp.pivot('metric', 'lambda_clp', 'performance')

temp = df_summary. \
            groupby(['outcome', 'sensitive_variable', 'metric', 'lambda_clp'])[['performance']]. \
            agg(lambda x: x). \
            reset_index()

temp = temp. \
        groupby(['outcome', 'sensitive_variable']). \
        apply(lambda x: x.pivot('metric', 'lambda_clp', 'performance'))

temp = temp[temp.index.get_level_values('metric').isin(['auc', 'auprc', 'brier', 'clp'])]

display(temp)

lambda_clp                              0.00      0.01      0.10      1.00   \
outcome   sensitive_variable metric                                           
los       age                auc     0.839281  0.839440  0.837327  0.838033   
                             auprc   0.564808  0.567654  0.564719  0.571301   
                             brier   0.120617  0.120257  0.120731  0.119884   
                             clp     0.225489  0.133754  0.046074  0.003846   
          gender             auc     0.843952  0.843763  0.842846  0.839208   
                             auprc   0.574549  0.571456  0.571536  0.568054   
                             brier   0.118158  0.118805  0.118338  0.118249   
                             clp     0.187826  0.097410  0.023434  0.001006   
          race_eth           auc     0.843057  0.842709  0.843434  0.844095   
                             auprc   0.560884  0.562984  0.565923  0.564842   
                             brier   0.119468  0.118931  0.119440  0.117250   
                             clp     0.049226  0.024830  0.013380  0.002392   
mortality age                auc     0.880798  0.862880  0.824412  0.830903   
                             auprc   0.196143  0.188137  0.192614  0.178008   
                             brier   0.023675  0.024598  0.023232  0.022684   
                             clp     0.935487  0.115688  0.015889  0.002908   
          gender             auc     0.879194  0.881849  0.879239  0.872823   
                             auprc   0.213159  0.220426  0.227778  0.212946   
                             brier   0.020705  0.020635  0.020426  0.020695   
                             clp     0.059450  0.002717  0.000712  0.000139   
          race_eth           auc     0.852125  0.858226  0.840942  0.837537   
                             auprc   0.194335  0.200944  0.189388  0.184679   
                             brier   0.025370  0.024139  0.023938  0.023816   
                             clp     0.348700  0.049575  0.015139  0.002144   

lambda_clp                              10.00  
outcome   sensitive_variable metric            
los       age                auc     0.836906  
                             auprc   0.569843  
                             brier   0.119177  
                             clp     0.000078  
          gender             auc     0.838614  
                             auprc   0.571090  
                             brier   0.117402  
                             clp     0.000054  
          race_eth           auc     0.844360  
                             auprc   0.562695  
                             brier   0.119624  
                             clp     0.000190  
mortality age                auc     0.826910  
                             auprc   0.184182  
                             brier   0.022230  
                             clp     0.000096  
          gender             auc     0.836618  
                             auprc   0.175058  
                             brier   0.023253  
                             clp     0.000004  
          race_eth           auc     0.794256  
                             auprc   0.192726  
                             brier   0.022243  
                             clp     0.000359

### baseline

In [143]:
performance_path_baseline = os.path.join(data_path, 'performance', 'baseline_default')

files_baseline = glob.glob(os.path.join(performance_path_baseline, '**', '*_eval.csv'), recursive=True)
df_baseline = pd.concat({(file, path_split(file)[-3], path_split(file)[-2]): 
                        pd.read_csv(file) for file in files_baseline})
df_baseline = df_baseline.rename_axis(index = ['file_path', 'outcome', 'experiment_id', 'index'])
df_baseline = df_baseline.reset_index([0, 2, 3], drop = True)
df_baseline = df_baseline.loc[df_baseline.phase == 'test']
df_baseline = df_baseline.drop(columns = ['epoch', 'phase'])
df_baseline = df_baseline.rename(columns = {'performance' : 'baseline'})
display(df_baseline)

,metric,baseline
outcome,,
los,auc,0.850835
los,auprc,0.581952
los,brier,0.115485
los,loss,0.365094
mortality,auc,0.892904
mortality,auprc,0.267086
mortality,brier,0.020567
mortality,loss,0.094960


In [162]:
temp

lambda_clp                              0.00      0.01      0.10      1.00   \
outcome   sensitive_variable metric                                           
los       age                auc     0.839281  0.839440  0.837327  0.838033   
                             auprc   0.564808  0.567654  0.564719  0.571301   
                             brier   0.120617  0.120257  0.120731  0.119884   
                             clp     0.225489  0.133754  0.046074  0.003846   
          gender             auc     0.843952  0.843763  0.842846  0.839208   
                             auprc   0.574549  0.571456  0.571536  0.568054   
                             brier   0.118158  0.118805  0.118338  0.118249   
                             clp     0.187826  0.097410  0.023434  0.001006   
          race_eth           auc     0.843057  0.842709  0.843434  0.844095   
                             auprc   0.560884  0.562984  0.565923  0.564842   
                             brier   0.119468  0.118931  0.119440  0.117250   
                             clp     0.049226  0.024830  0.013380  0.002392   
mortality age                auc     0.880798  0.862880  0.824412  0.830903   
                             auprc   0.196143  0.188137  0.192614  0.178008   
                             brier   0.023675  0.024598  0.023232  0.022684   
                             clp     0.935487  0.115688  0.015889  0.002908   
          gender             auc     0.879194  0.881849  0.879239  0.872823   
                             auprc   0.213159  0.220426  0.227778  0.212946   
                             brier   0.020705  0.020635  0.020426  0.020695   
                             clp     0.059450  0.002717  0.000712  0.000139   
          race_eth           auc     0.852125  0.858226  0.840942  0.837537   
                             auprc   0.194335  0.200944  0.189388  0.184679   
                             brier   0.025370  0.024139  0.023938  0.023816   
                             clp     0.348700  0.049575  0.015139  0.002144   

lambda_clp                              10.00  
outcome   sensitive_variable metric            
los       age                auc     0.836906  
                             auprc   0.569843  
                             brier   0.119177  
                             clp     0.000078  
          gender             auc     0.838614  
                             auprc   0.571090  
                             brier   0.117402  
                             clp     0.000054  
          race_eth           auc     0.844360  
                             auprc   0.562695  
                             brier   0.119624  
                             clp     0.000190  
mortality age                auc     0.826910  
                             auprc   0.184182  
                             brier   0.022230  
                             clp     0.000096  
          gender             auc     0.836618  
                             auprc   0.175058  
                             brier   0.023253  
                             clp     0.000004  
          race_eth           auc     0.794256  
                             auprc   0.192726  
                             brier   0.022243  
                             clp     0.000359

In [173]:
df = pd.merge(temp.reset_index([0, 1, 2]), df_baseline.reset_index(), how = 'left')
df = df.set_index(['outcome', 'sensitive_variable', 'metric'])
df = df[['baseline', 0.0, 0.01, 0.1, 1.0, 10.0]]
display(df)

baseline       0.0      0.01       0.1  \
outcome   sensitive_variable metric                                           
los       age                auc     0.850835  0.839281  0.839440  0.837327   
                             auprc   0.581952  0.564808  0.567654  0.564719   
                             brier   0.115485  0.120617  0.120257  0.120731   
                             clp          NaN  0.225489  0.133754  0.046074   
          gender             auc     0.850835  0.843952  0.843763  0.842846   
                             auprc   0.581952  0.574549  0.571456  0.571536   
                             brier   0.115485  0.118158  0.118805  0.118338   
                             clp          NaN  0.187826  0.097410  0.023434   
          race_eth           auc     0.850835  0.843057  0.842709  0.843434   
                             auprc   0.581952  0.560884  0.562984  0.565923   
                             brier   0.115485  0.119468  0.118931  0.119440   
                             clp          NaN  0.049226  0.024830  0.013380   
mortality age                auc     0.892904  0.880798  0.862880  0.824412   
                             auprc   0.267086  0.196143  0.188137  0.192614   
                             brier   0.020567  0.023675  0.024598  0.023232   
                             clp          NaN  0.935487  0.115688  0.015889   
          gender             auc     0.892904  0.879194  0.881849  0.879239   
                             auprc   0.267086  0.213159  0.220426  0.227778   
                             brier   0.020567  0.020705  0.020635  0.020426   
                             clp          NaN  0.059450  0.002717  0.000712   
          race_eth           auc     0.892904  0.852125  0.858226  0.840942   
                             auprc   0.267086  0.194335  0.200944  0.189388   
                             brier   0.020567  0.025370  0.024139  0.023938   
                             clp          NaN  0.348700  0.049575  0.015139   

                                          1.0      10.0  
outcome   sensitive_variable metric                      
los       age                auc     0.838033  0.836906  
                             auprc   0.571301  0.569843  
                             brier   0.119884  0.119177  
                             clp     0.003846  0.000078  
          gender             auc     0.839208  0.838614  
                             auprc   0.568054  0.571090  
                             brier   0.118249  0.117402  
                             clp     0.001006  0.000054  
          race_eth           auc     0.844095  0.844360  
                             auprc   0.564842  0.562695  
                             brier   0.117250  0.119624  
                             clp     0.002392  0.000190  
mortality age                auc     0.830903  0.826910  
                             auprc   0.178008  0.184182  
                             brier   0.022684  0.022230  
                             clp     0.002908  0.000096  
          gender             auc     0.872823  0.836618  
                             auprc   0.212946  0.175058  
                             brier   0.020695  0.023253  
                             clp     0.000139  0.000004  
          race_eth           auc     0.837537  0.794256  
                             auprc   0.184679  0.192726  
                             brier   0.023816  0.022243  
                             clp     0.002144  0.000359

In [175]:
table_path = './latex_tables'
os.makedirs(table_path, exist_ok = True)
with open(os.path.join(table_path, 'group_performance.txt'), 'w') as fp:
    df.to_latex(fp, float_format = '%.3g')

### by group

In [259]:
files_by_group = glob.glob(os.path.join(performance_path, '**', '*_by_group.csv'), recursive=True)
df_by_group = pd.concat({(file, path_split(file)[-4], path_split(file)[-3], path_split(file)[-2]): 
                        pd.read_csv(file) for file in files_by_group})
df_by_group = df_by_group.drop(columns = 'sensitive_variable')
df_by_group = df_by_group.rename_axis(index = ['file_path', 'outcome', 'sensitive_variable', 'experiment_id', 'index'])
df_by_group = df_by_group.reset_index([0, 1, 2, 3])
df_by_group['experiment_id'] = df_by_group['experiment_id'].astype(int)

In [260]:
df_by_group.head()

,file_path,outcome,sensitive_variable,experiment_id,group,metric,phase,epoch,performance
index,,,,,,,,,
0,data/performance/final_classifier_grid/los/age...,los,age,0,0,auc,val,0,0.887418
1,data/performance/final_classifier_grid/los/age...,los,age,0,0,auprc,val,0,0.642110
2,data/performance/final_classifier_grid/los/age...,los,age,0,0,brier,val,0,0.098240
3,data/performance/final_classifier_grid/los/age...,los,age,0,0,classification,val,0,0.314940
4,data/performance/final_classifier_grid/los/age...,los,age,0,0,loss,val,0,0.314940


In [261]:
df_config_temp = df_config.loc[(df_config.cf_gradients == True) &
                               (df_config.lambda_final_classifier_cf == 1.0) &
                               (df_config.lr_final_classifier == 1e-3)
                              ]
df_summary = pd.merge(df_by_group, df_config_temp, 
                      how = 'inner', 
                      on = ['experiment_id', 'outcome', 'sensitive_variable'])

df_summary = df_summary.loc[df_summary.phase == 'test']

temp = df_summary. \
            groupby(['outcome', 'sensitive_variable', 'metric', 'group', 'lambda_clp'])[['performance']]. \
            agg(lambda x: x). \
            reset_index()

temp = temp. \
        groupby(['outcome', 'sensitive_variable', 'group']). \
        apply(lambda x: x.pivot('metric', 'lambda_clp', 'performance'))

temp = temp[temp.index.get_level_values('metric').isin(['auc', 'auprc', 'brier', 'clp'])]

display(temp)

lambda_clp                                    0.00      0.01      0.10   \
outcome   sensitive_variable group metric                                 
los       age                0     auc     0.879818  0.869902  0.862532   
                                   auprc   0.622410  0.614965  0.588737   
                                   brier   0.098488  0.102041  0.104650   
                             1     auc     0.869006  0.867848  0.866198   
                                   auprc   0.549133  0.557163  0.545393   
                                   brier   0.090981  0.090550  0.091069   
                             2     auc     0.838456  0.838895  0.839321   
                                   auprc   0.587777  0.584016  0.579474   
                                   brier   0.129876  0.130763  0.128887   
                             3     auc     0.801292  0.802314  0.804272   
                                   auprc   0.545188  0.543022  0.543656   
                                   brier   0.139621  0.140060  0.137398   
          gender             0     auc     0.857860  0.854711  0.858148   
                                   auprc   0.546623  0.545872  0.549558   
                                   brier   0.102268  0.103734  0.101725   
                             1     auc     0.821605  0.818918  0.820695   
                                   auprc   0.597596  0.587262  0.593759   
                                   brier   0.137794  0.140394  0.139108   
          race_eth           0     auc     0.858979  0.854103  0.854307   
                                   auprc   0.569011  0.561557  0.567149   
                                   brier   0.109006  0.109905  0.109847   
                             1     auc     0.832389  0.824361  0.828701   
                                   auprc   0.561438  0.579652  0.564357   
                                   brier   0.133445  0.136933  0.131055   
                             2     auc     0.856558  0.857043  0.858321   
                                   auprc   0.566298  0.567555  0.565204   
                                   brier   0.116228  0.115596  0.115180   
                             3     auc     0.826442  0.819261  0.822535   
                                   auprc   0.562765  0.559122  0.554780   
                                   brier   0.125211  0.127206  0.123738   
...                                             ...       ...       ...   
mortality age                2     auc     0.893729  0.887527  0.879660   
                                   auprc   0.193092  0.186073  0.195420   
                                   brier   0.025545  0.024975  0.024441   
                             3     auc     0.824133  0.817023  0.820711   
                                   auprc   0.221712  0.223560  0.207116   
                                   brier   0.041352  0.041228  0.040440   
          gender             0     auc     0.915158  0.917028  0.914608   
                                   auprc   0.201268  0.232769  0.211753   
                                   brier   0.016291  0.015854  0.016394   
                             1     auc     0.841171  0.840329  0.833431   
                                   auprc   0.214159  0.223173  0.217891   
                                   brier   0.026540  0.026588  0.027380   
          race_eth           0     auc     0.850070  0.831725  0.752967   
                                   auprc   0.187665  0.167949  0.192417   
                                   brier   0.026274  0.026180  0.024400   
                             1     auc     0.872047  0.849300  0.862642   
                                   auprc   0.165003  0.234444  0.241105   
                                   brier   0.021893  0.022496  0.021146   
                             2     auc     0.866770  0.843100  0.821480   
                                   auprc   0.279531  0.258592  0.261596   
                         

In [262]:
## Load the baseline data
performance_path_baseline = os.path.join(data_path, 'performance', 'baseline_default')

files_baseline = glob.glob(os.path.join(performance_path_baseline, '**', '*_by_group.csv'), recursive=True)
df_baseline = pd.concat({(file, path_split(file)[-3], path_split(file)[-2]): 
                        pd.read_csv(file) for file in files_baseline})
df_baseline = df_baseline.rename_axis(index = ['file_path', 'outcome', 'experiment_id', 'index'])
df_baseline = df_baseline.reset_index([0, 1, 2, 3])
df_baseline = df_baseline.loc[df_baseline.phase == 'test']
df_baseline = df_baseline.drop(columns = ['epoch', 'phase'])
df_baseline = df_baseline.rename(columns = {'performance' : 'baseline'})
df_baseline = df_baseline.drop(columns = ['experiment_id', 'file_path', 'index'])
display(df_baseline)

,outcome,sensitive_variable,group,metric,baseline
4,los,age,0,auc,0.885375
5,los,age,0,auprc,0.607936
6,los,age,0,brier,0.098003
7,los,age,0,loss,0.345627
12,los,age,1,auc,0.881584
13,los,age,1,auprc,0.544652
14,los,age,1,brier,0.086962
15,los,age,1,loss,0.294602
20,los,age,2,auc,0.848664
21,los,age,2,auprc,0.606433


In [263]:
temp = pd.merge(temp.reset_index(), df_baseline).set_index(['outcome', 'sensitive_variable', 'group', 'metric'])

In [264]:
temp

0.0      0.01       0.1  \
outcome   sensitive_variable group metric                                 
los       age                0     auc     0.879818  0.869902  0.862532   
                                   auprc   0.622410  0.614965  0.588737   
                                   brier   0.098488  0.102041  0.104650   
                             1     auc     0.869006  0.867848  0.866198   
                                   auprc   0.549133  0.557163  0.545393   
                                   brier   0.090981  0.090550  0.091069   
                             2     auc     0.838456  0.838895  0.839321   
                                   auprc   0.587777  0.584016  0.579474   
                                   brier   0.129876  0.130763  0.128887   
                             3     auc     0.801292  0.802314  0.804272   
                                   auprc   0.545188  0.543022  0.543656   
                                   brier   0.139621  0.140060  0.137398   
          gender             0     auc     0.857860  0.854711  0.858148   
                                   auprc   0.546623  0.545872  0.549558   
                                   brier   0.102268  0.103734  0.101725   
                             1     auc     0.821605  0.818918  0.820695   
                                   auprc   0.597596  0.587262  0.593759   
                                   brier   0.137794  0.140394  0.139108   
          race_eth           0     auc     0.858979  0.854103  0.854307   
                                   auprc   0.569011  0.561557  0.567149   
                                   brier   0.109006  0.109905  0.109847   
                             1     auc     0.832389  0.824361  0.828701   
                                   auprc   0.561438  0.579652  0.564357   
                                   brier   0.133445  0.136933  0.131055   
                             2     auc     0.856558  0.857043  0.858321   
                                   auprc   0.566298  0.567555  0.565204   
                                   brier   0.116228  0.115596  0.115180   
                             3     auc     0.826442  0.819261  0.822535   
                                   auprc   0.562765  0.559122  0.554780   
                                   brier   0.125211  0.127206  0.123738   
...                                             ...       ...       ...   
mortality age                2     auc     0.893729  0.887527  0.879660   
                                   auprc   0.193092  0.186073  0.195420   
                                   brier   0.025545  0.024975  0.024441   
                             3     auc     0.824133  0.817023  0.820711   
                                   auprc   0.221712  0.223560  0.207116   
                                   brier   0.041352  0.041228  0.040440   
          gender             0     auc     0.915158  0.917028  0.914608   
                                   auprc   0.201268  0.232769  0.211753   
                                   brier   0.016291  0.015854  0.016394   
                             1     auc     0.841171  0.840329  0.833431   
                                   auprc   0.214159  0.223173  0.217891   
                                   brier   0.026540  0.026588  0.027380   
          race_eth           0     auc     0.850070  0.831725  0.752967   
                                   auprc   0.187665  0.167949  0.192417   
                                   brier   0.026274  0.026180  0.024400   
                             1     auc     0.872047  0.849300  0.862642   
                                   auprc   0.165003  0.234444  0.241105   
                                   brier   0.021893  0.022496  0.021146   
                             2     auc     0.866770  0.843100  0.821480   
                                   auprc   0.279531  0.258592  0.261596   
                                   brier   0.023423  0.024155  0.022360  

In [265]:
## Load group maps
label_path = os.path.join(data_path, 'labels')
os.listdir(label_path)
pd.read_csv(os.path.join(label_path, 'race_eth_map.csv'))
group_map_df = pd.concat(
    {key: pd.read_csv(os.path.join(label_path, '{}_map.csv'.format(key)))
          for key in ['age', 'gender', 'race_eth']}). \
        reset_index(1, drop = True). \
        rename_axis(index = 'sensitive_variable'). \
        rename(columns = {'category_id' : 'group', 'categories' : 'group_name'})

In [266]:
group_map_df

,group,group_name
sensitive_variable,,
age,0,"[18, 30)"
age,1,"[30, 45)"
age,2,"[45, 65)"
age,3,"[65, 89)"
gender,0,Female
gender,1,Male
gender,2,Other
race_eth,0,Asian
race_eth,1,Black


In [267]:
by_group_df = pd.merge(temp.reset_index(), group_map_df.reset_index(), how = 'left'). \
    set_index(['outcome', 'sensitive_variable', 'group_name', 'metric']). \
    drop(columns = 'group')
by_group_df = by_group_df[['baseline', 0.0, 0.01, 0.1, 1.0, 10.0]]

with open(os.path.join(table_path, 'by_group_performance.txt'), 'w') as fp:
    by_group_df.to_latex(fp, float_format = '%.3g')

temp = by_group_df.loc[by_group_df.index.get_level_values('outcome') == 'los']
temp = temp.reset_index(0, drop = True)
with open(os.path.join(table_path, 'by_group_performance_los.txt'), 'w') as fp:
    temp.to_latex(fp, float_format = '%.3g')

temp = by_group_df.loc[by_group_df.index.get_level_values('outcome') == 'mortality']
temp = temp.reset_index(0, drop = True)
with open(os.path.join(table_path, 'by_group_performance_mortality.txt'), 'w') as fp:
    temp.to_latex(fp, float_format = '%.3g')